In [1]:
import pandas as pd
import sys
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Set options to run Chrome in 'Headless' mode
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920x1080")


In [2]:
def start_driver_and_open_url(url):
    """
    Starts a webdriver and opens the given URL.

    Parameters
    ----------
    url: `str`
        String of the URL to retrieve image information.

    Returns
    -------
    driver : WebDriver
        Google webdriver at a predetermined URL.
    """

    #This installs the latest version of the official Google chromedriver
    #Accesses cached version if present.

    #Initialize variable
    driver = None

    #Opens driver. Installs if not present.
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),\
                options=chrome_options)
    except Exception as err:
        print(f"'Setting Driver Error: {err}'")
        sys.exit(0)

    #Set URL
    driver.get(url)
    
    return driver

This creates a dictionary of lists containing all the image names and image URLs found on the webpage.

If the image name has an extension of `.png`, then it is pulled into the dictionary.

If the image has an associated link, then it is pulled into the dictionary.

In [3]:
def get_image_info(driver):
    """
    This function retrieves all images from a specified webdriver.
    
    Parameters
    ----------
    driver : WebDriver
        Google webdriver at a predetermined URL.

    Returns
    -------
    image_dict : `dict`
        Dictionary containing image information such as the name of the image 
        and the link to the image.    
    """

    image_dict = {"name":[],"image_Link":[]}
    try:
        list_of_images = driver.find_elements(By.TAG_NAME,"img")
        for images in list_of_images:
            #Get Image Name
            if '.png' in images.get_attribute("alt"):
                image_dict['name'].append(images.get_attribute("alt"))
            #Get Image Link
            if images.get_attribute("data-src") == None:
                continue
            else:
                image_dict['image_Link'].append(images.get_attribute("data-src"))
    except Exception as e:
        print(e)
    
    return image_dict

Now that we have the image names and image links, we must match the image names to the image links.

To do this, we need to search each link for the given image name.

However, there are a few things we need to address and fix:

    1. The image link contains the image name, with the exception that the name now has underscores vs. spaces.
    2. The image size is not consistent (preference is 250 pixels).

In [4]:
def link_resize(image_Link_list, size = '250'):
    """
    Function to resize an image given it's URL from the DayZ Wiki.

    Parameters
    ----------
    image_Link_list : `list`
        A list containing url links to an image on the DayZ Wiki.

    size: `string`, optional
        Value of the size of the new image.
        Default resize value is 250 pixels.

    Returns
    -------
    new_link_list : `list`
        A list containing url links to an image on the DayZ Wiki
        adjusted by the `size` parameter.
    """

    new_link_list = []

    for links in image_Link_list:
        #Focus on section of link that needs to be changed
        string_index = links.find('down/')
        end_of_link = links[string_index+5:]
        #Split link by character to isolate picture size value
        temp_list = end_of_link.split('?')
        #Set new size to 250 pixels
        temp_list[0] = size
        #Combine new size with remaining portion of the image link
        new_link_end = temp_list[0] + '?' + temp_list[1]
        #Replace the old size with new size
        new_link = links.replace(end_of_link,new_link_end)
        new_link_list.append(new_link)

    return new_link_list

Now we will match on the adjusted name and append the actual item name and image link to the dictionary.

In [5]:
def match_name_to_link(image_dict):
    """
    Used to match image names to their corresponding updated links in
    a given image dictionary.

    Parameters
    ----------
    image_dict : `dict`
        Dictionary containing image information such as the name of the image 
        and the link to the image.

    Returns
    -------
    final_pair: `dict`
        Dictionary containing image name and the corresponding updated link.
    """
    final_pair = {"item_Name":[],"image_Link":[]}
    for names,adjusted_names in zip(image_dict['name'],image_dict['adjusted_name']):
        for links in image_dict['adjusted_links']:
            if adjusted_names in links:
                final_pair['item_Name'].append(names)
                final_pair['image_Link'].append(links)

    return final_pair

We also need to add the category of the items to the dictionary.

We will derive the category from the URL.

In [6]:

def get_item_category(url):
    """
    Retrieve the category from the given URL and input as a column
    in the given dictionary.

    Parameters
    ----------
    url: `str`
        String of the URL to retrieve image information.

    Returns
    -------
    image_category: `str`
        Category of the item.
    """

    #Split the URL and grab the category
    image_category = url.split('/')[-1]

    return image_category


Create final dataframe for the items on the page

In [7]:
def create_item_frame(final_pair,url):
    """
    Create final dataframe.

    Parameters
    ----------
    final_pair: `dict`
        Dictionary containing image name and the corresponding updated link.

    Returns
    -------
    item_df: `DataFrame`
        DataFrame containing image information from the given URL and dictionary.
    """

    #Turn the dictionary into a data frame
    item_df = pd.DataFrame(final_pair)
    #Set the column
    item_df['Category'] = get_item_category(url)
    #Fix item names
    item_df['item_Name'] = [x.replace('.png','') for x in item_df['item_Name']]

    return item_df
    

Use this to grab images from pages on the DayZ Wiki.

In [8]:
def execute_item_scrape(driver,url):
    """
    Master execution function.
    Grab all image information from a given URL.

    Parameters
    ----------
    driver: WebDriver
        The WebDriver instance to use.

    url: `str`
        String of the URL to fetch image information from.
    
    Returns
    -------
    item_df: `DataFrame`
        DataFrame containing image information from the given URL and dictionary.
    """

    #Grab image info
    image_dict = get_image_info(driver)
    #Adjust image names
    image_dict['adjusted_name'] = [name.replace(' ','_') for name in image_dict['name']]
    #Resize images
    image_dict['adjusted_links'] = link_resize(image_dict['image_Link'])
    #Rematch image names to thier links
    final_pair = match_name_to_link(image_dict)
    #Create final frame
    item_df = create_item_frame(final_pair,url)

    return item_df

In [9]:
def get_slot_size(driver,item_Name_with_png):
    """
    Retrieve the slot size for the given item.
    
    Parameters
    ----------
    driver : WebDriver
        The WebDriver instance to use.

    item_Name_with_png: `str`
        The name of the item to retrieve the slot size for.
        This must have the .png extension attached.

    Returns
    -------
    slot_size: `int`
        The slot size for the item.
    """

    item_Name = item_Name_with_png.replace('.png','')
    driver.get(f'https://dayz.fandom.com/wiki/{item_Name}')
    

    


In [97]:
driver = start_driver_and_open_url('https://dayz.fandom.com/wiki/Category:Food')

In [98]:
a_refs = driver.find_elements(By.TAG_NAME,"a")

In [99]:
start_point_index = 0

# Insert the name of the first item found on the page we
# want to scrape
first_item_in_list = 'Apple'

# This is the number of links/items to include in the scrape.
top_range = 5

for links in a_refs:
    start_point_index = start_point_index + 1
    if first_item_in_list in links.get_attribute('href'):
        start_point_index = start_point_index - 1
        end_point_index = start_point_index + top_range
        break


In [101]:
new_refs = a_refs[start_point_index:end_point_index]
new_refs_links = []
for links in new_refs:
    site = links.get_attribute('href')
    new_refs_links.append(site)



In [103]:
for links in new_refs_links:
    driver.get(links)
    slot_elements = driver.find_elements(By.TAG_NAME,"TBODY")
    item_detail = slot_elements[0].get_attribute('innerText').split('\n')
    slot_info_index = [item_detail.index(element) for element in item_detail if 'Slot' in element][0]
    slot_size_text = item_detail[slot_info_index].replace('(','').replace(')','')
    slot_size = [int(s) for s in slot_size_text.split() if s.isdigit()][-1]
    print(slot_size)


1
2
4
2
2


In [27]:
list_of_images = driver.find_elements(By.TAG_NAME,"img")

In [30]:
list_of_images[4].get_attribute('title')

''

In [ ]:
for images in list_of_images:
    #Get Image Name
    if '.png' in images.get_attribute("alt"):

In [ ]:
list_of_images = driver.find_elements(By.TAG_NAME,"img")
for images in list_of_images:
    #Get Image Name
    if '.png' in images.get_attribute("alt"):
        image_dict['name'].append(images.get_attribute("alt"))
    #Get Image Link
    if images.get_attribute("data-src") == None:
        continue
    else:
        image_dict['image_Link'].append(images.get_attribute("data-src"))
except Exception as e:

In [23]:
image_dict['slot_Size'] = [get_slot_size(driver,item) for item in image_dict['adjusted_name']]

UnboundLocalError: local variable 'item_Name' referenced before assignment

In [9]:
driver = start_driver_and_open_url('https://dayz.fandom.com')

In [10]:
url_dictionary = {'ammunition_url':'https://dayz.fandom.com/wiki/Ammunition',
'attachment_url':'https://dayz.fandom.com/wiki/Attachments',
'backpack_url':'https://dayz.fandom.com/wiki/Backpack',
'clothing_url':'https://dayz.fandom.com/wiki/Clothing',
'equipment_url':'https://dayz.fandom.com/wiki/Equipment',
'food_url':'https://dayz.fandom.com/wiki/Food_and_Drink',
'magazine_url':'https://dayz.fandom.com/wiki/Magazines',
'medical_url':'https://dayz.fandom.com/wiki/Medical_Supplies',
'resources_url':'https://dayz.fandom.com/wiki/Resources',
'weapons_url':'https://dayz.fandom.com/wiki/Weapons'
}

In [11]:
df_name = []
df_list = []
for url_names in url_dictionary.keys():
    #Access the URL
    url_link = url_dictionary.get(url_names)
    driver.get(url_link)
    df_name.append(f'{url_names}_image_df')
    df_list.append(execute_item_scrape(driver,url_link))
    


In [12]:
master_df = pd.DataFrame()

In [13]:
for frames in df_list:
    master_df = pd.concat([master_df,frames])

In [14]:
master_df = master_df.drop_duplicates(
    subset = ['item_Name','image_Link'],
    keep = 'first').reset_index(drop = True)

In [15]:
master_df = master_df.drop_duplicates().reset_index(drop = True)

In [16]:
master_df.to_csv('dayz_images_info.csv',header=True,index=False)

In [17]:
    master_df

,item_Name,image_Link,Category
0,Sporter 22 Wood,https://static.wikia.nocookie.net/dayz_gameped...,Ammunition
1,15rd Sporter 22 Mag,https://static.wikia.nocookie.net/dayz_gameped...,Ammunition
2,30 rnd 22 mag,https://static.wikia.nocookie.net/dayz_gameped...,Ammunition
3,Ammo 380,https://static.wikia.nocookie.net/dayz_gameped...,Ammunition
4,AmmoBox 380 35Rnd,https://static.wikia.nocookie.net/dayz_gameped...,Ammunition
...,...,...,...
512,Plastic Explosive,https://static.wikia.nocookie.net/dayz_gameped...,Weapons
513,Improvised Explosive,https://static.wikia.nocookie.net/dayz_gameped...,Weapons
514,LandMine,https://static.wikia.nocookie.net/dayz_gameped...,Weapons
515,RGD-5 Grenade,https://static.wikia.nocookie.net/dayz_gameped...,Weapons
